## 1) Typy w Haskellu: *type vs. newtype*

### 1.5.1

In [1]:
newtype CartesianCoord a = MkCartesianCoord (a, a) deriving (Show)
newtype PolarCoord a = MkPolarCoord (a, a) deriving (Show)
  
polarToCartesian :: Floating a => PolarCoord a -> CartesianCoord a
polarToCartesian (MkPolarCoord (r, phi)) = MkCartesianCoord (r * cos phi, r * sin phi)

In [2]:
okType = MkPolarCoord (1, pi)
res1 = polarToCartesian okType

In [3]:
res1

MkCartesianCoord (-1.0,1.2246467991473532e-16)

In [4]:
wrongType = (1, 2)
res2 = polarToCartesian wrongType

: 

In [5]:
wrongType2 = MkCartesianCoord (1, 2)
res3 = polarToCartesian wrongType2

: 

Taka implementacja pozwala na łatwe odróżnienie koordynatów kartezjańskich i polarnych; gwarantuje, że nie zaaplikujemy funkcji do argumentów złego typu - za każdym razem, kiedy zechcemy utworzyć punkt o koordynatach polarnych będziemy musieli to robić świadomie korzystajac z `MkPolarCoord`

## 2) Algebraiczne typy danych 1: product & sum types, record syntax

### 2.7.1

In [6]:
data Cart3DVec a = Cart3DVec a a a

xCoord3D (Cart3DVec x _ _) = x
yCoord3D (Cart3DVec _ y _) = y
zCoord3D (Cart3DVec _ _ z) = z

coord1 = Cart3DVec 1 2 3

xCoord3D coord1
yCoord3D coord1
zCoord3D coord1

data Cart3DVec' a = Cart3DVec' {x::a, y::a, z::a}

coord2 = Cart3DVec' 4 2 0

x coord2
y coord2
z coord2

1

2

3

4

2

0

### 2.7.6

In [7]:
data Shape = Circle Float | Rectangle Float Float
area :: Shape -> Float
area (Circle r) = pi * r * r
area (Rectangle a b) = a * b

shape1 = Circle $ sqrt 2
shape2 = Rectangle 4 2

area shape1
area shape2

6.2831855

8.0

### 2.7.7

In [8]:
data TrafficLights = Red | RedYellow | Yellow | Green
actionFor :: TrafficLights -> String

actionFor Red = "hold on"
actionFor RedYellow = "launch control"
actionFor Yellow = "pedal to the metal or you won't make it"
actionFor Green = "good to go"

actionFor Red
actionFor Yellow

"hold on"

"pedal to the metal or you won't make it"

## 3) Algebraiczne typy danych 2: rekursja strukturalna

### 3.4.1

In [9]:
data BinTree a = EmptyBT | NodeBT a (BinTree a) (BinTree a)

instance (Show a) => Show (BinTree a) where
  show EmptyBT = "∅"
  show (NodeBT n lt rt) = "{" ++ show lt ++ "<-" ++ show n ++ "->" ++ show rt ++ "}"

tree = NodeBT 2 (NodeBT 2 EmptyBT EmptyBT) (NodeBT 3 EmptyBT EmptyBT)

In [10]:
depthOfBT :: BinTree a -> Int
depthOfBT EmptyBT = 0
depthOfBT (NodeBT _ lt rt) = 1 + max (depthOfBT lt) (depthOfBT rt)

flattenBTPre :: BinTree a -> [a]
flattenBTPre EmptyBT = []
flattenBTPre (NodeBT n lt rt) = [n] ++ flattenBTPre lt ++ flattenBTPre rt

flattenBTIn :: BinTree a -> [a]
flattenBTIn EmptyBT = []
flattenBTIn (NodeBT n lt rt) = flattenBTIn lt ++ [n] ++ flattenBTIn rt

flattenBTPost :: BinTree a -> [a]
flattenBTPost EmptyBT = []
flattenBTPost (NodeBT n lt rt) = flattenBTPost lt ++ flattenBTPost rt  ++ [n]

mapBT :: (a -> b) -> BinTree a -> BinTree b
mapBT _ EmptyBT = EmptyBT
mapBT f (NodeBT n lt rt) = NodeBT (f n) (mapBT f lt) (mapBT f rt)

insert :: Ord a => a -> BinTree a -> BinTree a
insert n EmptyBT = NodeBT n EmptyBT EmptyBT
insert n (NodeBT curr lt rt)
  | n <= curr = NodeBT curr (insert n lt) rt
  | otherwise = NodeBT curr lt (insert n rt)

import Data.List
list2BST :: Ord a => [a] -> BinTree a
list2BST list = internal $ sort list
  where internal [] = EmptyBT
        internal xs = NodeBT m (internal l) (internal r)
          where (l, (m:r)) = splitAt (length xs `div` 2) xs

In [11]:
flattenBTPre $ list2BST [1, 2, 3, 4, 5, 6, 7]
flattenBTIn $ list2BST [1, 2, 3, 4, 5, 6, 7]
flattenBTPost $ list2BST [1, 2, 3, 4, 5, 6, 7]
flattenBTPre $ insert 1 tree
insert 5 $ insert 1 $ insert 2 tree
list2BST [7, 6, 5, 4, 3, 2, 1]

[4,2,1,3,6,5,7]

[1,2,3,4,5,6,7]

[1,3,2,5,7,6,4]

[2,2,1,3]

{{{{∅<-1->∅}<-2->∅}<-2->∅}<-2->{∅<-3->{∅<-5->∅}}}

{{{∅<-1->∅}<-2->{∅<-3->∅}}<-4->{{∅<-5->∅}<-6->{∅<-7->∅}}}

### 3.4.6

In [12]:
data Expr a = Lit a | 
              Add (Expr a) (Expr a) |
              Sub (Expr a) (Expr a) |
              Mul (Expr a) (Expr a)

eval :: Num a => Expr a -> a
eval (Lit n) = n
eval (Add e1 e2) = eval e1 + eval e2
eval (Sub e1 e2) = eval e1 - eval e2
eval (Mul e1 e2) = eval e1 * eval e2

instance (Show a) => Show (Expr a) where
  show (Lit n) = show n
  show (Add e1 e2) = "(" ++ show e1 ++ "+" ++ show e2 ++ ")"
  show (Sub e1 e2) = "(" ++ show e1 ++ "-" ++ show e2 ++ ")"
  show (Mul e1 e2) = "(" ++ show e1 ++ "*" ++ show e2 ++ ")"

In [13]:
expr = Sub (Mul (Add (Lit 3) (Lit 5)) (Sub (Lit 3) (Lit 1))) (Lit 10)
expr
eval expr

(((3+5)*(3-1))-10)

6

## 5) Klasy typów i ich instancje 1: dołączanie typu do istniejącej klasy

### 5.6.1

In [14]:
newtype MyInt = MkMyInt Int

In [15]:
MkMyInt 1 == MkMyInt 1

: 

In [16]:
instance Eq MyInt where
  (==) (MkMyInt i1) (MkMyInt i2) = i1 == i2

In [17]:
MkMyInt 1 == MkMyInt 1
MkMyInt 1 == MkMyInt 2
MkMyInt 1 /= MkMyInt 2
MkMyInt 1 < MkMyInt 2

True

False

True

: 

In [18]:
instance Ord MyInt where
  (<=) (MkMyInt i1) (MkMyInt i2) = i1 <= i2

In [19]:
MkMyInt 1 <= MkMyInt 2
MkMyInt 1 < MkMyInt 2
MkMyInt 1 > MkMyInt 2
MkMyInt 1 >= MkMyInt 2

:i Ord

MkMyInt 1 + MkMyInt 2
:i Num

True

True

False

False

: 

class Eq a => Ord a where
  compare :: a -> a -> Ordering
  (<) :: a -> a -> Bool
  (<=) :: a -> a -> Bool
  (>) :: a -> a -> Bool
  (>=) :: a -> a -> Bool
  max :: a -> a -> a
  min :: a -> a -> a
  {-# MINIMAL compare | (<=) #-}
  	-- Defined in ‘GHC.Classes’
instance [safe] Ord MyInt -- Defined at <interactive>:1:10
instance (Ord a, Ord b) => Ord (Either a b) -- Defined in ‘Data.Either’
instance Ord IHaskellTH.TypeFamilyHead -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Type -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.TyVarBndr -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.TySynEqn -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.TyLit -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Stmt -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.SourceUnpackedness -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.SourceStrictness -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Safety -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.RuleMatch -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.RuleBndr -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Role -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Range -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Pragma -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Phases -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.PatSynDir -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.PatSynArgs -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Pat -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Overlap -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.NameSpace -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Name -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.ModuleInfo -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Match -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Loc -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Lit -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Inline -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.InjectivityAnn -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Info -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Guard -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.FunDep -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Foreign -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.FixityDirection -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Fixity -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.FamilyResultSig -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Exp -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.DerivStrategy -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.DerivClause -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.DecidedStrictness -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Dec -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Con -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Clause -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Callconv -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Body -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.Bang -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.AnnTarget -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellTH.AnnLookup -- Defined in ‘Language.Haskell.TH.Syntax’
instance Ord IHaskellSysIO.NewlineMode -- Defined in ‘GHC.IO.Handle.Types’
instance Ord IHaskellSysIO.Newline -- Defined in ‘G

In [20]:
instance Num MyInt where
  (+) (MkMyInt i1) (MkMyInt i2) = MkMyInt (i1 + i2)
  (-) (MkMyInt i1) (MkMyInt i2) = MkMyInt (i1 - i2)
  (*) (MkMyInt i1) (MkMyInt i2) = MkMyInt (i1 * i2)
  negate (MkMyInt i)            = MkMyInt (negate i)
  abs (MkMyInt i)               = MkMyInt (abs i)
  signum (MkMyInt i)            = MkMyInt (signum i)
  fromInteger int               = MkMyInt (fromIntegral int)

In [21]:
MkMyInt 1 + MkMyInt 2

MkMyInt 1
:i Show

: 

In [22]:
instance Show MyInt where
  show (MkMyInt i) = "MkMyInt " ++ show i

In [23]:
MkMyInt 1 + MkMyInt 2
MkMyInt 5
(MkMyInt 2) * (MkMyInt 3 + MkMyInt 4)

(MkMyInt 5) ^ 2
MkMyInt 1 `div` MkMyInt 2

Line 3: Redundant bracket
Found:
(MkMyInt 2) * (MkMyInt 3 + MkMyInt 4)
Why not:
MkMyInt 2 * (MkMyInt 3 + MkMyInt 4)Line 5: Redundant bracket
Found:
(MkMyInt 5) ^ 2
Why not:
MkMyInt 5 ^ 2

MkMyInt 3

MkMyInt 5

MkMyInt 14

MkMyInt 25

: 

### 5.6.2

Trochę kusi, żeby zrobić `(==) a b = show a == show b` ;-)

In [24]:
instance Eq a => Eq (BinTree a) where
  (==) EmptyBT EmptyBT = True
  (==) EmptyBT _ = False
  (==) _ EmptyBT = False
  (==) (NodeBT n1 lt1 rt1) (NodeBT n2 lt2 rt2) = n1 == n2 && lt1 == lt2 && rt1 == rt2

In [25]:
a = list2BST [2, 1, 3]
b = list2BST [3, 2, 1]
c = list2BST [3, 3, 2]
a
b
c
a == b
a == c
b == c

{{∅<-1->∅}<-2->{∅<-3->∅}}

{{∅<-1->∅}<-2->{∅<-3->∅}}

{{∅<-2->∅}<-3->{∅<-3->∅}}

True

False

False

## 7) Moduły i importy

### 7.6.1

Przećwiczone poza Jupyterem (w GHCI)